In [1]:
import os
from glob import glob 

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

In [2]:
# 더 적은 쪽을 기준으로 가져온다 
tmp_label_paths = glob('ADE20K_2017_05_30_consistency/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k/*_seg.png')

image_paths = []
label_paths = []

for lbl_path in tmp_label_paths:
    img_path = lbl_path.replace('_seg.png', '.jpg')
    if os.path.exists(img_path):
        image_paths.append(img_path)
        label_paths.append(lbl_path)
        
len(image_paths), len(label_paths)

(64, 64)

In [3]:
class_nums = np.arange(13)
class_nums

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# Preprare Dataset

In [4]:
def data_preprocess(img_path, lbl_path):
    image_tf = tf.io.read_file(img_path)
    image = tf.io.decode_jpeg(image_tf) / 255
    image = tf.image.resize(image, (512, 512))
    
    label_tf = tf.io.read_file(lbl_path)
    label = tf.io.decode_png(label_tf)

    label_c, _ = tf.split(label, [1, -1], axis=-1)
    label_c = tf.image.resize(label_c, [512, 512], 'nearest')
    label = tf.squeeze(label_c)    
    label_onehot = tf.one_hot(label, len(class_nums))
    return image, label_onehot

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
dataset = dataset.map(data_preprocess)